# EWM(지수 이동평균) : Exponentially Weighted Moving Average

## 1. EWM(지수이동평균)_직접 구현

In [37]:
import FinanceDataReader as fdr

df = fdr.DataReader('005930',start='2021')
df['Close']

Date
2021-01-04    83000
2021-01-05    83900
2021-01-06    82200
2021-01-07    82900
2021-01-08    88800
              ...  
2025-01-20    53400
2025-01-21    53500
2025-01-22    54300
2025-01-23    53700
2025-01-24    53700
Name: Close, Length: 1000, dtype: int64

In [38]:
df = df[['Close']]
df


,Close
Date,
2021-01-04,83000
2021-01-05,83900
2021-01-06,82200
2021-01-07,82900
2021-01-08,88800
...,...
2025-01-20,53400
2025-01-21,53500
2025-01-22,54300


In [39]:
N = 9
weight = 2 /(N+1)
print(weight)

0.2


In [40]:
# df['EWM'] = float(df['Close'].iloc[0])

# for index in range(len(df['EWM']) - 1): 
#     df['EWM'].iloc[index+1] = df['Close'].iloc[index+1]*weight + df['Close'].iloc[index]*(1-weight)

In [41]:
### [판다스 버전 수정] : iloc를 이용해 데이터프레임에 값 할당을 지양하라 -> 리스트를 만들어서 값을 넣는 것을 권장함!

df['EWM'] = float(df['Close'].iloc[0]) # float으로 값 변경! 

# 새로운 Series 생성 후 최종적으로 DataFrame에 할당
ewm_values = [df['Close'].iloc[0]]  # 첫 번째 값을 리스트에 추가
for index in range(1, len(df)):  # 첫 번째 값은 이미 추가했으므로 1부터 시작
    new_value = df['Close'].iloc[index] * weight + ewm_values[-1] * (1 - weight)
    ewm_values.append(new_value)

# 계산된 결과를 DataFrame에 다시 할당
df['EWM'] = ewm_values


In [42]:
df

,Close,EWM
Date,,
2021-01-04,83000,83000.000000
2021-01-05,83900,83180.000000
2021-01-06,82200,82984.000000
2021-01-07,82900,82967.200000
2021-01-08,88800,84133.760000
...,...,...
2025-01-20,53400,54171.082108
2025-01-21,53500,54036.865686
2025-01-22,54300,54089.492549


In [43]:
df.loc['2021-12-10']

Close    76900.00000
EWM      76180.19076
Name: 2021-12-10 00:00:00, dtype: float64

In [44]:
def ewm_fn(df,window):
    
    df = df.copy()
    weight = 2 /(window+1)

    df['EWM'] = float(df['Close'].iloc[0])  

    ewm_values = [df['Close'].iloc[0]]  
    for index in range(1, len(df)):  
        new_value = df['Close'].iloc[index] * weight + ewm_values[-1] * (1 - weight)
        ewm_values.append(new_value)
    
    df['EWM'] = ewm_values

    return df[['EWM']]

In [45]:
df_ewm = ewm_fn(df,window=9)
df_ewm

,EWM
Date,
2021-01-04,83000.000000
2021-01-05,83180.000000
2021-01-06,82984.000000
2021-01-07,82967.200000
2021-01-08,84133.760000
...,...
2025-01-20,54171.082108
2025-01-21,54036.865686
2025-01-22,54089.492549


## 2. EWM(지수이동평균)_pandas.DataFrame.ewm

- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html
- `adjust = False`


In [46]:
df = fdr.DataReader('005930',start='2021')
df=df[['Close']]
df 

,Close
Date,
2021-01-04,83000
2021-01-05,83900
2021-01-06,82200
2021-01-07,82900
2021-01-08,88800
...,...
2025-01-20,53400
2025-01-21,53500
2025-01-22,54300


In [47]:
short_N = 9
long_N = 26
signal_N = 13

df['Short'] = df['Close'].ewm(span=short_N,adjust=False).mean()
df['Long'] = df['Close'].ewm(span=long_N,adjust=False).mean()

df['MACD'] = df['Short'] - df['Long']

df['Signal'] = df['MACD'].ewm(span=signal_N,adjust=False).mean()
df['MACD Oscillator'] = df['MACD'] - df['Signal']

df

,Close,Short,Long,MACD,Signal,MACD Oscillator
Date,,,,,,
2021-01-04,83000,83000.000000,83000.000000,0.000000,0.000000,0.000000
2021-01-05,83900,83180.000000,83066.666667,113.333333,16.190476,97.142857
2021-01-06,82200,82984.000000,83002.469136,-18.469136,11.239103,-29.708239
2021-01-07,82900,82967.200000,82994.878829,-27.678829,5.679398,-33.358228
2021-01-08,88800,84133.760000,83424.887805,708.872195,106.135512,602.736683
...,...,...,...,...,...,...
2025-01-20,53400,54171.082108,54542.648182,-371.566074,-229.563855,-142.002219
2025-01-21,53500,54036.865686,54465.414983,-428.549297,-257.990347,-170.558950
2025-01-22,54300,54089.492549,54453.162022,-363.669473,-273.087365,-90.582108


In [49]:
def macdOscillator_fn(df,short_N,long_N,signal_N):

    df=df[['Close']].copy()

    df['Short'] = df['Close'].ewm(span= short_N, adjust=False).mean()
    df['Long'] = df['Close'].ewm(span= long_N, adjust=False).mean()

    df['MACD'] = df['Short'] - df['Long']

    df['Signal'] = df['MACD'].ewm(span=signal_N, adjust=False).mean()

    df['MACD Oscillator'] = df['MACD'] - df['Signal']

    return df[['MACD','Signal','MACD Oscillator']]


In [50]:
df = fdr.DataReader('005930',start='2021')

macdOscillator_fn(df=df,short_N=9,long_N=26,signal_N=13)

,MACD,Signal,MACD Oscillator
Date,,,
2021-01-04,0.000000,0.000000,0.000000
2021-01-05,113.333333,16.190476,97.142857
2021-01-06,-18.469136,11.239103,-29.708239
2021-01-07,-27.678829,5.679398,-33.358228
2021-01-08,708.872195,106.135512,602.736683
...,...,...,...
2025-01-20,-371.566074,-229.563855,-142.002219
2025-01-21,-428.549297,-257.990347,-170.558950
2025-01-22,-363.669473,-273.087365,-90.582108
